### **1. Paramètres**

In [6]:
# Importation des bibliothèques
import os

# Importation des fonctions
from functions_masquage import (add_band_names, create_vrt, clip_and_align_raster, mask_vrt, verifier_raster)

In [7]:
# Définition des années et des zones d'intérêt
annees = [2017, 2018, 2019, 2020, 2021, 2022]
zones = ['T30TYP', 'T30TYQ', 'T31TCJ', 'T31TGL', 'T31UEP']

# Liste des métriques LiDAR à traiter
lidar_metrics = ['grid_CC', 'grid_ENL', 'grid_MOCH', 'grid_PAI', 'grid_VCI']

# Définition des chemins vers les répertoires de données
base_dir = "../data_brut/raster"
output_confidence_temp = "../data_brut/raster/stacks/confidence"  # Répertoire temporaire pour les VRT de confiance
output_lidar_temp = "../data_brut/raster/stacks/lidar"           # Répertoire temporaire pour les rasters LiDAR clipés
output_confidence_final = "../data_final/raster/confidence"      # Répertoire final pour les rasters de confiance masqués
output_lidar_final = "../data_final/raster/lidar"                # Répertoire final pour les rasters LiDAR masqués

# Chemin vers le GeoPackage utilisé pour le masquage
gpkg_path = '../data_final/vector/peupleraies_lidar_parcelle.gpkg'

### **2. Créer les piles des rasters de confidence**

In [8]:
# 2.1 Création des VRT pour les rasters de confiance
for zone in zones:
    create_vrt(base_dir, zone, annees, 'confidence', output_confidence_temp)

VRT créé à : ../data_brut/raster/stacks/confidence\confidence_stack_T30TYP.vrt
VRT créé à : ../data_brut/raster/stacks/confidence\confidence_stack_T30TYQ.vrt
VRT créé à : ../data_brut/raster/stacks/confidence\confidence_stack_T31TCJ.vrt
VRT créé à : ../data_brut/raster/stacks/confidence\confidence_stack_T31TGL.vrt
VRT créé à : ../data_brut/raster/stacks/confidence\confidence_stack_T31UEP.vrt


In [9]:
# 2.2 Masquage des VRT de confiance et sauvegarde au format TIFF
for zone in zones:
    vrt_path = os.path.join(output_confidence_temp, f'confidence_stack_{zone}.vrt')
    output_raster = mask_vrt(
        vrt_path=vrt_path,
        gpkg_path=gpkg_path,
        output_dir=output_confidence_final,
        output_name=f"confidence_masked_{zone}",
        nodata_value=0,
        dtype_value='int16'
    )
    
    # Ajout des noms des bandes correspondant aux années
    add_band_names(output_raster, [str(annee) for annee in annees])

Raster masqué enregistré à : ../data_final/raster/confidence\confidence_masked_T30TYP.tif
Descriptions ajoutées pour le raster : ../data_final/raster/confidence\confidence_masked_T30TYP.tif
Raster masqué enregistré à : ../data_final/raster/confidence\confidence_masked_T30TYQ.tif
Descriptions ajoutées pour le raster : ../data_final/raster/confidence\confidence_masked_T30TYQ.tif
Raster masqué enregistré à : ../data_final/raster/confidence\confidence_masked_T31TCJ.tif
Descriptions ajoutées pour le raster : ../data_final/raster/confidence\confidence_masked_T31TCJ.tif
Raster masqué enregistré à : ../data_final/raster/confidence\confidence_masked_T31TGL.tif
Descriptions ajoutées pour le raster : ../data_final/raster/confidence\confidence_masked_T31TGL.tif
Raster masqué enregistré à : ../data_final/raster/confidence\confidence_masked_T31UEP.tif
Descriptions ajoutées pour le raster : ../data_final/raster/confidence\confidence_masked_T31UEP.tif


### **3. Créer la pile des métriques lidar**

In [10]:
# 3.1 Initialisation de la liste pour stocker les rasters LiDAR masqués
lidar_rasters_masked = []

# Création des répertoires de sortie si ils n'existent pas
os.makedirs(output_lidar_temp, exist_ok=True)
os.makedirs(output_lidar_final, exist_ok=True)

# Itération sur chaque zone pour traiter les métriques LiDAR
for zone in zones:
    if zone == "T31UEP":  # Ignorer la tuile sans métriques LiDAR
        continue
    
    # Chemin vers le raster de référence (confiance masquée) pour la zone actuelle
    reference_mask_path = os.path.join(output_confidence_final, f"confidence_masked_{zone}.tif")
    
    for metric in lidar_metrics:
        input_raster = os.path.join(base_dir, f"{metric}.tif")
        output_clipped = os.path.join(output_lidar_temp, f"{metric}_clipped_{zone}.tif")
        
        if os.path.exists(input_raster):
            # 3.1.1 Aligner et clipper le raster LiDAR
            clip_and_align_raster(input_raster, reference_mask_path, output_clipped)
            
            # 3.1.2 Masquer le raster LiDAR aligné et clipé
            output_masked = mask_vrt(
                vrt_path=output_clipped,
                gpkg_path=gpkg_path,
                output_dir=output_lidar_final,
                output_name=f"{metric}_masked_{zone}",
                nodata_value=-999,
                dtype_value='float32'
            )
            
            # Ajouter le raster LiDAR masqué à la liste
            lidar_rasters_masked.append(output_masked)
        else:
            print(f" Fichier LiDAR manquant : {input_raster} pour la tuile {zone}")


Raster clipé et aligné sauvegardé : ../data_brut/raster/stacks/lidar\grid_CC_clipped_T30TYP.tif
Raster masqué enregistré à : ../data_final/raster/lidar\grid_CC_masked_T30TYP.tif
Raster clipé et aligné sauvegardé : ../data_brut/raster/stacks/lidar\grid_ENL_clipped_T30TYP.tif
Raster masqué enregistré à : ../data_final/raster/lidar\grid_ENL_masked_T30TYP.tif
Raster clipé et aligné sauvegardé : ../data_brut/raster/stacks/lidar\grid_MOCH_clipped_T30TYP.tif
Raster masqué enregistré à : ../data_final/raster/lidar\grid_MOCH_masked_T30TYP.tif
Raster clipé et aligné sauvegardé : ../data_brut/raster/stacks/lidar\grid_PAI_clipped_T30TYP.tif
Raster masqué enregistré à : ../data_final/raster/lidar\grid_PAI_masked_T30TYP.tif
Raster clipé et aligné sauvegardé : ../data_brut/raster/stacks/lidar\grid_VCI_clipped_T30TYP.tif
Raster masqué enregistré à : ../data_final/raster/lidar\grid_VCI_masked_T30TYP.tif
Raster clipé et aligné sauvegardé : ../data_brut/raster/stacks/lidar\grid_CC_clipped_T30TYQ.tif
Rast

### **4. Vérification des raster générés**

In [11]:
# 4.1 Définition des chemins des rasters de confiance à vérifier
stack_confidence = [
    os.path.join(output_confidence_final, 'confidence_masked_T30TYP.tif'),
    os.path.join(output_confidence_final, 'confidence_masked_T30TYQ.tif'),
    os.path.join(output_confidence_final, 'confidence_masked_T31TCJ.tif'),
    os.path.join(output_confidence_final, 'confidence_masked_T31TGL.tif'),
    os.path.join(output_confidence_final, 'confidence_masked_T31UEP.tif')
]

In [12]:
# 4.2 Définition du chemin du raster LiDAR à vérifier
lidar = lidar_rasters_masked  # Liste des rasters LiDAR masqués

In [13]:
# 4.3 Vérification des statistiques et des valeurs NoData pour chaque raster de confiance
for raster in stack_confidence:
    verifier_raster(raster)   

=== Vérification du Raster: ../data_final/raster/confidence\confidence_masked_T30TYP.tif ===
Nombre de bandes: 6
Valeur NoData: 0.0
Résolution: (10.0, 10.0)
Projection (CRS): EPSG:2154
--- Bande 1 ---
  Pixels NoData (0): 35165122
  Pixels valides: 58
  Min: 24
  Max: 64
  Moyenne: 41.10344827586207
--- Bande 2 ---
  Pixels NoData (0): 35165110
  Pixels valides: 70
  Min: 18
  Max: 95
  Moyenne: 54.24285714285714
--- Bande 3 ---
  Pixels NoData (0): 35165134
  Pixels valides: 46
  Min: 19
  Max: 71
  Moyenne: 44.608695652173914
--- Bande 4 ---
  Pixels NoData (0): 35165046
  Pixels valides: 134
  Min: 19
  Max: 100
  Moyenne: 67.71641791044776
--- Bande 5 ---
  Pixels NoData (0): 35165043
  Pixels valides: 137
  Min: 20
  Max: 100
  Moyenne: 79.75912408759125
--- Bande 6 ---
  Pixels NoData (0): 35164963
  Pixels valides: 217
  Min: 21
  Max: 100
  Moyenne: 80.02304147465438
=== Vérification du Raster: ../data_final/raster/confidence\confidence_masked_T30TYQ.tif ===
Nombre de bandes: 6

In [14]:
# 4.4 Vérification des statistiques et des valeurs NoData pour chaque raster LiDAR
for raster in lidar:
    verifier_raster(raster)

=== Vérification du Raster: ../data_final/raster/lidar\grid_CC_masked_T30TYP.tif ===
Nombre de bandes: 1
Valeur NoData: -999.0
Résolution: (10.0, 10.0)
Projection (CRS): EPSG:2154
--- Bande 1 ---
  Pixels NoData (0): 35164299
  Pixels valides: 881
  Min: 0.0
  Max: 96.8499984741211
  Moyenne: 50.25421142578125
=== Vérification du Raster: ../data_final/raster/lidar\grid_ENL_masked_T30TYP.tif ===
Nombre de bandes: 1
Valeur NoData: -999.0
Résolution: (10.0, 10.0)
Projection (CRS): EPSG:2154
--- Bande 1 ---
  Pixels NoData (0): 35164299
  Pixels valides: 881
  Min: 0.0
  Max: 22.5
  Moyenne: 13.141316413879395
=== Vérification du Raster: ../data_final/raster/lidar\grid_MOCH_masked_T30TYP.tif ===
Nombre de bandes: 1
Valeur NoData: -999.0
Résolution: (10.0, 10.0)
Projection (CRS): EPSG:2154
--- Bande 1 ---
  Pixels NoData (0): 35164299
  Pixels valides: 881
  Min: 0.0
  Max: 26.58850860595703
  Moyenne: 9.941807746887207
=== Vérification du Raster: ../data_final/raster/lidar\grid_PAI_masked_